### Data preparation for visual analysis

In [2]:
print("_"*100)
print("Dataset and libraries are being downloaded ...") 

import numpy as np
import pandas as pd 

# We'll need this for 'Job Description' and 'Job Title'
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')

# N-gram counting
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings("ignore")


df = pd.read_csv('DataScientist.csv', na_values = -1)


print("_"*100)
print("Dataset is ready for processing ...") 


# Split "Location" into State and City
df[['City', 'State']] = df.Location.str.split(', ', expand=True)  
df = df[:][df.State != 'United Kingdom'].reset_index()
print("_"*100)
print("Deleting vacancies outside USA ...")


# New variable
df['job_in_head_office'] = np.where(df.Location == df.Headquarters, 1, 0)
print("_"*100)
print('Variable "job_in_head_office" is created ...')


# Salary range 
hours_per_week = 40
weeks_per_year = 52

for i in range(df.shape[0]):
    salary_estimate = df.loc[i,"Salary Estimate"]
    salary_estimate = salary_estimate.replace("$", "")
    
    if "Per Hour" in salary_estimate:
        lower, upper = salary_estimate.split("-")
        upper, _ = upper.split("Per")
        upper= upper.strip()
        lower = int(lower)*hours_per_week*weeks_per_year*(1/1000)
        upper = int(upper)*hours_per_week*weeks_per_year*(1/1000)
        
    else:
        lower, upper = salary_estimate.split("-")
        lower = lower.replace("K", "")
        upper, _= upper.split("(")
        upper=upper.replace("K", "")
        upper = upper.strip()
    
        
    lower = int(lower)
    upper = int(upper)
    df.loc[i,"salary_estimate_lower_bound"] = lower
    df.loc[i,"salary_estimate_upper_bound"] = upper
print("_"*100)
print('Variables "salary_estimate_lower_bound" and "salary_estimate_lower_bound" created ...')


#job title
df['Title'] = 0
for i in range(len(df['Job Title'])):
    
    jobtitle = df['Job Title'][i].lower()
    
    if 'data scientist' in jobtitle or 'data science' in jobtitle:
        df['Title'][i] = 'data scientist' 
        
    elif 'data analyst' in jobtitle or 'data analytics' in jobtitle:
        df['Title'][i] = 'data analyst'
        
    elif 'data engineer' in jobtitle or 'engineer data' in jobtitle:
        df['Title'][i] = 'data engineer'
        
    elif 'machine learning' in jobtitle:
        df['Title'][i] = 'machine learning specialist'
        
    elif 'research scientist' in jobtitle:
        df['Title'][i] = 'research scientist'
        
    elif 'big data' in jobtitle or 'big data engineer' in jobtitle:
        df['Title'][i] = 'big data engineer'
    
    elif 'learning engineer' in jobtitle or 'machine learning engineer' in jobtitle:
        df['Title'][i] = 'machine learning engineer'
    
    elif 'business intelligence' in jobtitle or 'business intelligence analyst' in jobtitle or  'intelligence analyst' in jobtitle or 'bi analyst' in jobtitle:
        df['Title'][i] = 'bi analyst'
        
    elif 'analytics manager' in jobtitle or 'product manager' in jobtitle:
        df['Title'][i] = 'analytics manager'
    
    elif 'data modeler' in jobtitle:
        df['Title'][i] = 'data modeler'
    
    elif 'laboratory scientist' in jobtitle or 'lab scientist' in jobtitle:
        df['Title'][i] = 'lab scientist'
    
    elif 'quantitative researcher' in jobtitle or 'quantum data' in jobtitle or 'quantum data scientist' in jobtitle:
        df['Title'][i] = 'quantum data scientist'
        
    elif 'data scientist industrial' in jobtitle or 'scientist industrial' in jobtitle or 'industrial process' in jobtitle:
        df['Title'][i] = 'data scientist industrial'
        
    elif 'data architect' in jobtitle:
        df['Title'][i] = 'data architect'
        
    elif 'medical lab scientist' in jobtitle or 'medical data scientist' in jobtitle or 'medical data' in jobtitle:
         df['Title'][i] = 'medical lab scientist'
    else:
        df['Title'][i] = 'data specialist'
print("_"*100)
print('Variables "Title" created ...')


# Does the vacancy related to science?
df['Research'] = np.where((df['Job Title'].str.lower().str.contains('research')|
                          df['Job Title'].str.lower().str.contains('doctoral')), 1, 0)

# Level of the specialist

junior = ['junior', 'jr', 'intern', 'internship']
senior = ['sr', 'senior']
lead = ['lead', 'teamlead', 'team lead']
director = ['director', 'principal', 'vp', 'head', 'manager']

df['job_level'] = 'Not stated'
for i in range(df.shape[0]):
    for jun in junior: 
        if jun in df['Job Title'][i].lower():
            df['job_level'][i] = 'junior'
            
    for sen in senior:
        if sen in df['Job Title'][i].lower():
            df['job_level'][i] = 'senior'
    
    for led in lead:
        if led in df['Job Title'][i].lower():
            df['job_level'][i] = 'lead'
            
    for direc in director:
        if direc in df['Job Title'][i].lower():
            df['job_level'][i] = 'head'   

print("_"*100)
print('"job_level" variable created ...')            

# Education
bachelor_degree = ['bs', 'bachelor']
master_degree = ['ms/ma', 'master']
phd_degree = ['phd', 'postgraduate', 'ph.d.']

# highest level of education required
df['level_of_education'] = 'Not stated'
for i in range(df.shape[0]):
    
    for bachelor in bachelor_degree: 
        if bachelor in df['Job Description'][i].lower():
            df['level_of_education'][i] = 'Bachelor'
            
    for master in master_degree:
        if master in df['Job Description'][i].lower():
            df['level_of_education'][i] = 'Master'
            
    for phd in phd_degree:
        if phd in df['Job Description'][i].lower():
            df['level_of_education'][i] = 'Phd'
print("_"*100)
print('Variable "level_of_education" created ...')

#Skills
skills_list = ['python', 
        'rstudio', 
        'tableau', 
        'power bi', 
        'powerbi',
        'java',
        'scala',
        'go',
        'bigquery',
        'hive',
        'javascript', 
        'c\+\+', 
        'julia', 
        'hadoop', 
        'sql', 
        'postgresql', 
        'nosql', 
        'neural networks', 
        'cnn', 
        'nlp', 
        'saas',
        'matlab',
        'pyspark',
        'tensorflow',
        'pytorch',
        'etl',
        'ms excel',
        'ms powerpoint',
        'powerpoint',
        'spark',
        'google analytics',
        'crm',
        'amazon web services',
        'sas']
# creating the binary variables for skills needed
for skill in skills_list:
    df[skill] = np.where(df['Job Description'].str.lower().str.contains(skill), 1, 0)   
print("_"*100)
print('Variables of skills needed created ...')

#Replacing the UNKNOWN values
df['Size'] = np.where(df.Size == 'Unknown', np.nan, df.Size)
df['Type of ownership'] = np.where(df['Type of ownership'] == 'Unknown', np.nan, df['Type of ownership'])
df['Revenue'] = np.where(df.Revenue == 'Unknown / Non-Applicable', np.nan, df.Revenue)
df['Rating'] = np.where(df.Rating < 2., 1, df['Rating'])
df['Rating'] = np.where(((df.Rating >= 2.) & (df.Rating < 3.)), 2, df['Rating'])
df['Rating'] = np.where(((df.Rating >= 3.) & (df.Rating < 4.)), 3, df['Rating'])
df['Rating'] = np.where(((df.Rating >= 4.) & (df.Rating < 4.5)), 4, df['Rating'])
df['Rating'] = np.where(df.Rating >= 4.5, 5, df['Rating'])
print("_"*100)
print('"Unknown" values replaced ...')


df['Company Name'] = df['Company Name'].str.partition('\n')[0]
df['Easy Apply'] = np.where(df['Easy Apply'] == True, 1, 0)
df.drop(['Job Description', 'Job Title'], axis = 1, inplace = True)
print("_"*100)
print('Dataset is ready ...')
print("_"*100)
print('Export dataset to csv-file ...')
df.to_csv(r'Jobs_in_USA_Tableau.csv', index = False)
print("_"*100)
print('Dataset exported ...')

____________________________________________________________________________________________________
Dataset and libraries are being downloaded ...
____________________________________________________________________________________________________
Dataset is ready for processing ...
____________________________________________________________________________________________________
Deleting vacancies outside USA ...
____________________________________________________________________________________________________
Variable "job_in_head_office" is created ...


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Valeria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


____________________________________________________________________________________________________
Variables "salary_estimate_lower_bound" and "salary_estimate_lower_bound" created ...
____________________________________________________________________________________________________
Variables "Title" created ...
____________________________________________________________________________________________________
"job_level" variable created ...
____________________________________________________________________________________________________
Variable "level_of_education" created ...
____________________________________________________________________________________________________
Variables of skills needed created ...
____________________________________________________________________________________________________
"Unknown" values replaced ...
____________________________________________________________________________________________________
Dataset is ready ...
____________